In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('/Users/brianoktavec/MSC550/Assignment_5/election2000-1.csv')

In [3]:
df.head()

,Bush,UnE,Pop,Male,Male18,Pop65,NonMet,Pov,NuHouse,Inc50,Inc75,Inc100
0,56.47,4.6,4447100,48.27,47.26,13.0,32.68,16.1,1472906,14.3,4.3,2.0
1,58.62,6.6,626932,51.69,51.83,5.7,58.48,9.4,85359,36.3,16.8,6.9
2,51.02,3.9,5130632,49.92,49.42,13.0,14.03,13.9,1571330,11.0,4.2,1.8
3,51.31,4.4,2673400,48.80,47.94,14.0,53.66,15.8,836388,10.0,2.9,1.6
4,41.65,4.9,33871648,49.82,49.26,10.6,3.31,14.2,9709296,23.1,10.0,4.4


# Q1

In [4]:
import statsmodels.api as sm
ols = sm.OLS(df.Bush, sm.add_constant(df.iloc[:, 1:11]))
lm1 = ols.fit()
print(lm1.summary())

                            OLS Regression Results                            
Dep. Variable:                   Bush   R-squared:                       0.813
Model:                            OLS   Adj. R-squared:                  0.760
Method:                 Least Squares   F-statistic:                     15.22
Date:                Sun, 05 Mar 2023   Prob (F-statistic):           5.50e-10
Time:                        23:00:52   Log-Likelihood:                -134.22
No. Observations:                  46   AIC:                             290.4
Df Residuals:                      35   BIC:                             310.6
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       -675.4612    185.928     -3.633      0.0

In [5]:
import scipy.stats as st
from scipy.stats import ttest_ind

for x in df:
    print(x + " Test:")
    y = ttest_ind(df['Bush'], df[x])
    print(y)
    print("")

Bush Test:
Ttest_indResult(statistic=0.0, pvalue=1.0)

UnE Test:
Ttest_indResult(statistic=29.69860623437394, pvalue=2.754334813145617e-48)

Pop Test:
Ttest_indResult(statistic=-5.948678482647366, pvalue=5.088806581071438e-08)

Male Test:
Ttest_indResult(statistic=0.5497033885302841, pvalue=0.5838843684253888)

Male18 Test:
Ttest_indResult(statistic=1.0056630785624672, pvalue=0.31727386313099276)

Pop65 Test:
Ttest_indResult(statistic=23.949258010632764, pvalue=8.139994909685252e-41)

NonMet Test:
Ttest_indResult(statistic=5.2388878406535975, pvalue=1.054811107239253e-06)

Pov Test:
Ttest_indResult(statistic=23.290710972317264, pvalue=7.090494658482932e-40)

NuHouse Test:
Ttest_indResult(statistic=-5.942635423159532, pvalue=5.225342710998258e-08)

Inc50 Test:
Ttest_indResult(statistic=16.940426523725858, pvalue=9.728991693820893e-30)

Inc75 Test:
Ttest_indResult(statistic=26.24299677077692, pvalue=6.012450156017872e-44)

Inc100 Test:
Ttest_indResult(statistic=29.98942657618159, pvalue=

# Q2

In [6]:
import itertools

def fit_lm(feature_set):
    ols = sm.OLS(df.Bush, sm.add_constant(df[list(feature_set)]))
    lm = ols.fit()
    RSS = ((lm.predict(sm.add_constant(df[list(feature_set)])) - df.Bush) ** 2).sum()
    rsquared_adj = lm.rsquared_adj
    AIC = lm.aic
    BIC = lm.bic
    return {"model":lm, "RSS":RSS, "Rsquared_adj": rsquared_adj, "AIC":AIC, "BIC": BIC}


def getbest(k):
    result = []
    for combo in itertools.combinations(df.columns[:-1], k):
        result.append(fit_lm(combo))
    models = pd.DataFrame(result)
    best_model = models.loc[models['AIC'].argmin()]
    return best_model

In [7]:
import time
models_best = pd.DataFrame(columns=["model","RSS","Rsquared_adj","AIC","BIC"])
start = time.time()
for i in range(1,11):
    models_best.loc[i] = getbest(i)
end = time.time()
print(models_best)
print("Best subset selection takes:", end-start, "seconds")
print(models_best.loc[2, "model"].summary())
lm2 = models_best.loc[2, "model"]

/Users/brianoktavec/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/linear_model.py:903: RuntimeWarning: divide by zero encountered in log
  llf = -nobs2*np.log(2*np.pi) - nobs2*np.log(ssr / nobs) - nobs2


                                                model           RSS  \
1   <statsmodels.regression.linear_model.Regressio...  6.588566e-27   
2   <statsmodels.regression.linear_model.Regressio...  2.934563e-28   
3   <statsmodels.regression.linear_model.Regressio...  0.000000e+00   
4   <statsmodels.regression.linear_model.Regressio...  1.782826e-27   
5   <statsmodels.regression.linear_model.Regressio...  3.070247e-27   
6   <statsmodels.regression.linear_model.Regressio...  5.064487e-27   
7   <statsmodels.regression.linear_model.Regressio...  8.813154e-27   
8   <statsmodels.regression.linear_model.Regressio...  3.655581e-26   
9   <statsmodels.regression.linear_model.Regressio...  1.125547e-23   
10  <statsmodels.regression.linear_model.Regressio...  5.915876e-18   

    Rsquared_adj          AIC          BIC  
1            1.0 -2814.660399 -2811.003117  
2            1.0 -2955.783072 -2950.297148  
3            1.0         -inf         -inf  
4            1.0 -2868.788666 -2859.64

# Q3

In [8]:
remain = list(df.columns[:-1])
picked = list()
models_best = pd.DataFrame(columns=["model","RSS","Rsquared_adj","AIC","BIC"])
start = time.time()
for i in range(1,12):
    best_RSS = np.inf
    for combo in itertools.combinations(remain,1):
        RSS = fit_lm(list(combo) + picked)["BIC"]   
        if RSS < best_RSS:
            best_RSS = RSS
            best_feature = combo[0]
            models_best.loc[i] = fit_lm(list(combo) + picked)
    picked.append(best_feature)
    remain.remove(best_feature)
end = time.time()
print(models_best)
print("Forward selection takes:", end-start, "seconds")
print(models_best.loc[10, "model"].summary())
lm3 = models_best.loc[10, "model"]

                                                model          RSS  \
1   <statsmodels.regression.linear_model.Regressio...  6.58857e-27   
2   <statsmodels.regression.linear_model.Regressio...  8.48814e-28   
3   <statsmodels.regression.linear_model.Regressio...    3.045e-27   
4   <statsmodels.regression.linear_model.Regressio...  7.50721e-22   
5   <statsmodels.regression.linear_model.Regressio...  2.10754e-19   
6   <statsmodels.regression.linear_model.Regressio...  2.15681e-20   
7   <statsmodels.regression.linear_model.Regressio...    1.689e-19   
8   <statsmodels.regression.linear_model.Regressio...  2.30129e-18   
9   <statsmodels.regression.linear_model.Regressio...  9.95852e-20   
10  <statsmodels.regression.linear_model.Regressio...  4.76497e-18   
11  <statsmodels.regression.linear_model.Regressio...  5.83735e-18   

   Rsquared_adj      AIC      BIC  
1             1 -2814.66    -2811  
2             1 -2906.93 -2901.44  
3             1 -2846.16 -2838.85  
4             1

# Q4

In [9]:
pick = list(df.columns[:-1])
models_best = pd.DataFrame(columns=["model","RSS","Rsquared_adj","AIC","BIC"])
models_best.loc[0] = fit_lm(pick)
start = time.time()
for i in range(1,11):
    best_RSS = np.inf
    for combo in itertools.combinations(pick,len(pick)-1):
        RSS = fit_lm(list(combo))["RSS"]   
        if RSS < best_RSS:
            best_RSS = RSS
            best_feature = combo
            models_best.loc[i] = fit_lm(list(combo))
    pick = best_feature
end = time.time()
print(models_best)
print("Backward selection takes:", end-start, "seconds")
print(models_best.loc[2, "model"].summary())
lm4 = models_best.loc[2, "model"]

                                                model          RSS  \
0   <statsmodels.regression.linear_model.Regressio...  5.40926e-17   
1                                               model          RSS   
2   <statsmodels.regression.linear_model.Regressio...  1.37235e-17   
3   <statsmodels.regression.linear_model.Regressio...  1.22162e-18   
4   <statsmodels.regression.linear_model.Regressio...  1.43436e-17   
5   <statsmodels.regression.linear_model.Regressio...  5.95595e-17   
6   <statsmodels.regression.linear_model.Regressio...  2.12291e-24   
7   <statsmodels.regression.linear_model.Regressio...  2.81276e-25   
8   <statsmodels.regression.linear_model.Regressio...  2.10058e-26   
9   <statsmodels.regression.linear_model.Regressio...  3.62908e-26   
10  <statsmodels.regression.linear_model.Regressio...  6.58857e-27   

    Rsquared_adj      AIC      BIC  
0              1 -1744.54  -1722.6  
1   Rsquared_adj      AIC      BIC  
2              1 -1811.64 -1793.35  
3          

# Q5 (BONUS)

In [10]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

cv = KFold(n_splits=10, random_state=1, shuffle=True)
mse = []

for i in range(1, 12):
    x = df[models_best.loc[i, "model"].params.index[1:]]
    y = df.iloc[:, 12]
    error = []
    
    for train_index, test_index in cv.split(df):
        x_train, x_test = x.iloc[train_index], x.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        ols = sm.OLS(y_train, sm.add_constant(x_train))
        lm = ols.fit()
        y_pred = lm.predict(sm.add_constant(x_test))
        error.append(mean_squared_error(y_pred,y_test))
    mse.append(np.mean(error))

AttributeError: 'str' object has no attribute 'params'

# Q6

In [11]:
df_new = pd.read_csv('/Users/brianoktavec/MSC550/Assignment_5/election2000_test.csv')

In [12]:
df_new.head()

,Bush,UnE,Pop,Male,Male18,Pop65,NonMet,Pov,NuHouse,Inc50,Inc75,Inc100
0,62.25,3.0,1711263,49.29,48.61,13.6,47.42,9.7,599558,15.3,5.4,2.2
1,46.43,4.2,12281054,48.29,47.34,15.6,15.39,11.0,4256258,17.3,5.7,2.3
2,40.70,2.9,608827,49.01,48.27,12.7,67.33,9.4,190533,19.3,6.4,3.1
3,45.50,3.3,4919479,49.52,48.88,12.1,29.60,7.9,1876964,19.3,6.1,2.9
4,35.23,4.6,18976457,48.21,47.20,12.9,7.92,14.6,5862138,26.6,12.9,7.0


In [13]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

# 1
# ols = sm.OLS(df_new.Bush, sm.add_constant(df_new.iloc[:, 1:12]))
# lm = ols.fit()
y_pred1 = lm1.predict(sm.add_constant(df_new.iloc[:, 1:11]))
mse1 = mean_squared_error(y_pred1,df_new.Bush)

# 2
y_pred2 = lm2.predict(sm.add_constant(df_new.iloc[:, 1:3]))
mse2 = mean_squared_error(y_pred2,df_new.Bush)

# 3
y_pred3 = lm3.predict(sm.add_constant(df_new.iloc[:, 1:11]))
mse3 = mean_squared_error(y_pred3,df_new.Bush)

# 4
y_pred4 = lm4.predict(sm.add_constant(df_new.iloc[:, 1:10]))
mse4 = mean_squared_error(y_pred4,df_new.Bush)
    

In [14]:
print("MSE for Model 1 is:", mse1)
print("MSE for Model 2 is:", mse2)
print("MSE for Model 3 is:", mse3)
print("MSE for Model 4 is:", mse4)

MSE for Model 1 is: 90.52406100908848
MSE for Model 2 is: 1888.3624601271772
MSE for Model 3 is: 107685476970996.97
MSE for Model 4 is: 1888.3637884867735


# Q7

In [15]:
data = {'model1':[0.760,298.4,310.6,90.524], 'model2':[1.00, -2956,-2950,1888.36], 'model3':[1.00, -1858,-1838, 107685476970996.97], 'model4':[1.00, -1812,-1793,1888.3637884867735]}
df_final = pd.DataFrame(data, index =['Adj.R^2', 'AIC', 'BIC', 'MSE']) 

In [16]:
print(df_final)

          model1   model2        model3       model4
Adj.R^2    0.760     1.00  1.000000e+00     1.000000
AIC      298.400 -2956.00 -1.858000e+03 -1812.000000
BIC      310.600 -2950.00 -1.838000e+03 -1793.000000
MSE       90.524  1888.36  1.076855e+14  1888.363788
